# Filtragem Baseada em Conteudo
## KNN

In [9]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
import math

import time

In [10]:
# Importando .csv
train = pd.read_csv('dataset/train_data.csv', header=None,  skiprows=[0], usecols=[0,1,2])
m = pd.read_csv('dataset/movies_data.csv')

# Obtendo generos dos filmes
m = pd.read_csv('dataset/movies_data.csv')
genres = pd.get_dummies(m.set_index(['movie_id']).genres.str.split('|', expand=True).stack(dropna=False)).sum(level=0)

In [11]:
# Função para cria modelo
def fbc_knn(train, features, k = 4):
    ratings = train.pivot(index=1, columns=0, values=2)
    ratings.fillna(0.0, inplace=True)
    sim = genres.T.corr(method=distance.jaccard)
    sim.fillna(0.0, inplace=True)

    return { "sim": sim, "K": k, "ratings": ratings }

In [12]:
# Função de predição
def predict(model, user, item, k = 5):
    sim = model["sim"]
    ratings = model["ratings"]
    if item not in sim or user not in ratings:
        return 0
    sim_items = sim[item].sort_values(ascending=False).index
    rated_items = ratings[user][ratings[user] > 0].index
    sim_k = np.intersect1d(sim_items, rated_items)
    top_k = []
    for x in sim_items:
        if k <= -1:
            break
        if x in sim_k:
            top_k.append(x)
            k-=1
    # sim_k = [x for x in sim_items if x in sim_k][:k]
    top_k = sim_k
    sumSim = 0.0
    sumWeight = 0.0
    for j in sim_k:
        sumSim += sim[item][j]
        sumWeight += sim[item][j] * ratings[user][j]
    if sumSim == 0.0:
        return 0

    return sumWeight/sumSim

In [13]:
# Avaliando predições
def rmse(model, test, k = 5):
    sum_err = 0
    for t in test:
        u = t[0]
        i = t[1]
        r_ui = t[2]
        pred = predict(model, u, i, k)
        error = (r_ui - pred)**2
        sum_err += error
    return math.sqrt(sum_err/len(test))

In [14]:
def validation(model, data, features,  train_split = 0.75, k = 5):
    data = data.sample(frac=1)
    train_len = int(train_split * len(data))
    train = data[:train_len]
    test = data[train_len:].values

    start_time = time.time()
    model = model(train, features)
    print("Tempo de treinamento em segundos: ", time.time() - start_time)
    start_time = time.time()
    print("RMSE :", rmse(model, test, k))
    print("Tempo de predição em segundos: ", time.time() - start_time)

In [15]:
# rmse(model, r.values[535000:])

In [16]:
features = genres.values
features = np.hstack((features,np.ones((len(features),1))))
validation(fbc_knn, train, features, 0.75, 15)

Tempo de treinamento em segundos:  158.92344737052917
RMSE : 1.0428914475503248
Tempo de predição em segundos:  2037.5774657726288


## Execução 1 treino 75% k = 10 (misturando todos os usuários)
Tempo de treinamento em segundos:  155.23047995567322 <br />
RMSE : 1.1807414506623164 <br />
Tempo de predição em segundos:  1679.8644750118256 <br />

## Execução 2 treino 93% k = 5 (misturando todos os usuários)
Tempo de treinamento em segundos:  164.93956518173218 <br />
RMSE : 1.046533140508873 <br />
Tempo de predição em segundos:  1085.375151872635 <br />

## Execução 3 treino 93% k = 10 (misturando todos os usuários)
Tempo de treinamento em segundos:  259.33242893218994 <br />
RMSE : 1.040549165141219 <br />
Tempo de predição em segundos:  810.3907425403595 <br />

## Execução 4 treino 75% k = 5 (misturando todos os usuários)
Tempo de treinamento em segundos:  180.85930705070496 <br />
RMSE : 1.042281858212767 <br />
Tempo de predição em segundos:  2381.4849865436554 <br />

## Execução 5 treino 75% k = 10 (misturando todos os usuários)
Tempo de treinamento em segundos:  210.99122667312622 <br />
RMSE : 1.0426094798929306 <br />
Tempo de predição em segundos:  2519.689395427704 <br />

## Execução 6 treino 75% k = 10 (misturando todos os usuários)
Tempo de treinamento em segundos:  179.80445313453674 <br />
RMSE : 1.0433772019897898 <br />
Tempo de predição em segundos:  1994.827300786972 <br />

## Execução 7 treino 75% k = 15 (misturando todos os usuários)
Tempo de treinamento em segundos:  158.92344737052917 <br />
RMSE : 1.0428914475503248 <br />
Tempo de predição em segundos:  2037.5774657726288 <br />